# Alunos:

##### - Frederick Campbell
##### - Lucas Ayub Rodrigues Tuler

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

---

# Rascunho

# 1. Carregamento dos dados

In [2]:
train = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('../input/airline-passenger-satisfaction/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../input/airline-passenger-satisfaction/train.csv'

In [ ]:
train.shape

In [ ]:
# Get a snapshot of data
train.head(10)

In [ ]:
df = pd.read_csv('airline_passenger_satisfaction.csv')  # nome do arquivo baixado do Kaggle
df.head()

# 2. Limpeza e pré-processamento inicial

In [ ]:
# Verificar valores nulos
print(df.isnull().sum())

# Remover entradas inválidas (se houver)
df = df.dropna()

# Resetar índice
df = df.reset_index(drop=True)


# 3. Análise Exploratória dos Dados (EDA)

In [ ]:
# Distribuição da variável alvo
sns.countplot(data=df, x='satisfaction')
plt.title('Distribuição da Satisfação dos Passageiros')
plt.show()


In [ ]:
# Distribuições demográficas
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.histplot(df['Age'], kde=True, ax=axs[0]).set_title("Distribuição de Idade")
sns.countplot(data=df, x='Gender', ax=axs[1]).set_title("Distribuição por Gênero")
sns.countplot(data=df, x='Customer Type', ax=axs[2]).set_title("Tipo de Cliente")
plt.tight_layout()
plt.show()

In [ ]:
# Relação entre classe e satisfação
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Class', hue='satisfaction')
plt.title('Satisfação por Classe de Voo')
plt.show()

In [ ]:
# Boxplots para outliers
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='satisfaction', y='Flight Distance')
plt.title('Distância do Voo vs Satisfação')
plt.show()

In [ ]:
# Heatmap de correlação (apenas colunas numéricas)
plt.figure(figsize=(12, 8))
sns.heatmap(df.select_dtypes(include=np.number).corr(), annot=True, cmap='coolwarm')
plt.title('Correlação entre variáveis numéricas')
plt.show()

# 4. Codificação de variáveis categóricas

In [ ]:
# Label Encoding para variável alvo
df['satisfaction'] = df['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})

# One-Hot Encoding para variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

# 5. Separação dos dados

In [ ]:
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# Normalização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão treino/validação
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# 6. Construção e treino do modelo MLP

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_val, y_val), callbacks=[early_stop])

# 7. Avaliação do modelo

In [ ]:
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.xlabel('Epoch')
plt.ylabel('Acurácia')
plt.legend()
plt.title('Curva de Acurácia')
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Curva de Perda')
plt.show()

In [ ]:
# Métricas
y_pred = (model.predict(X_val) > 0.5).astype(int)

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

# 8. Comparação com outro modelo (opcional)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred_rf = clf.predict(X_val)

print(classification_report(y_val, y_pred_rf))